In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import numpy as np
import string
import random
import string
from sklearn import linear_model
import pandas
from sklearn.metrics import mean_squared_error

def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

# Read data from file and return a generator of lines, each line is a dictionary
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
# read data from file and return a generator of user, game, and dictionary
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d
# 
def UsersGames(data):
    usergames = defaultdict(set)
    gameusers = defaultdict(set)
    for u, g, d in data:
        usergames[u].add(g)
        gameusers[g].add(u)
    return usergames, gameusers


# allData
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)


d:\SDK\python\anaconda\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\SDK\python\anaconda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\SDK\python\anaconda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
d:\SDK\python\anaconda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# split data into train and valid
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

augmentedValidationSet = []

valid1 = []
valid2 = []

answers = {}

allGames = set([g for u, g, d in hoursTrain + hoursValid]) # 所有gameid的集合

usergames,gameusers = UsersGames(hoursTrain + hoursValid) # usergames: 每个user玩过的game; gameusers:每个game的user


for u,i,d in hoursValid:
    valid1.append((u, i, 1))
    valid2.append((u, random.choice(list(allGames - usergames[u])), 0))
augmentedValidationSet = valid1 + valid2 # 加入一些用户没玩过的游戏作为负样本

In [3]:

gt = np.asarray([p for u, g, p in augmentedValidationSet])
user = np.asarray([u for u, g, p in augmentedValidationSet])
game = np.asarray([g for u, g, p in augmentedValidationSet])


val_user_count = np.asarray([len(gameusers[g]) for u, g, p in augmentedValidationSet])
pred = np.zeros(len(gt))
res_true = len(pred)/2 
for i in range(len(pred)): 
    if val_user_count[i] > 16:
        pred[i] = 1
    if pred[i] == 1:
        res_true -= 1
    if res_true == 0:
        break
acc = [1 if pred[i] == p else 0 for i, (u, g, p) in enumerate(augmentedValidationSet)]
print(sum(acc)/len(augmentedValidationSet))

0.983998399839984
